In [69]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
import local_python_packages.features_adding as local

In [2]:
pd.set_option('display.max_columns', None)
flights_data = pd.read_csv('UA flights 2019.csv',parse_dates=[0])
flights_data = flights_data.sort_values(['fl_date'])

In [3]:
df_weather = pd.read_csv('cities_and_dates_weather_final.csv',parse_dates=[1])

Adding additional columns from our feature engineering

In [4]:
flights_data = local.add_taxi_Ndays_rolling(flights_data, 30)

In [5]:
flights_data = local.add_traffic_rolling(flights_data, 30)

In [6]:
flights_data = local.make_month_dummies(flights_data, 'fl_date')

In [7]:
flights_data = local.merging_weather_flights(flights_data,df_weather)

In [8]:
flights_data = local.add_dep_delay_Ndays_rolling(flights_data, 30)

In [9]:
flights_data = local.make_dates_ordinal(flights_data, 'fl_date')

In [14]:
features_list = ['fl_date','taxi_out','taxi_in','arr_delay','crs_elapsed_time',
       'air_time', 'distance',
       '30d taxi_out', '30d taxi_in',
       '30d roll flts origin_airport_id', '30d roll flts dest_airport_id',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'origin_city_wspd', 'origin_visibility',
       'dest_city_wspd', 'dest_visibility', 'origin_cond_Overcast',
       'origin_cond_Partially cloudy', 'origin_cond_Rain', 'origin_cond_Snow',
       'dest_cond_Overcast', 'dest_cond_Partially cloudy', 'dest_cond_Rain',
       'dest_cond_Snow','30 days roll dep_time']

In [15]:
flights_data = local.distill_features(flights_data,desired_features=features_list)

In [16]:
flights_data.shape

(1571606, 36)

In [24]:
flights_data = local.replace_nan_with_mean(flights_data, 'arr_delay')

/Users/christiantruchsess/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4439: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [31]:
# features_list.remove('arr_delay')

In [33]:
flights_data.shape

(1571606, 36)

In [43]:
flights_data = flights_data.dropna()

In [44]:
flights_data.shape

(1514251, 36)

In [91]:
X = flights_data[features_list]
y = flights_data['arr_delay']

In [92]:
scaler = StandardScaler()

In [93]:
X = scaler.fit_transform(X)

In [94]:
X_train, X_test, y_train, y_test = local.quick_split(X,y,train_ratio=0.75)

In [95]:
linreg = LinearRegression()

In [96]:
linreg.fit(X_train,y_train)

LinearRegression()

In [97]:
y_pred = linreg.predict(X_test)

In [98]:
y_pred = y_pred.reshape(-1,1)

In [99]:
y_pred.shape

(378563, 1)

In [100]:
y_test = y_test.to_numpy().reshape(-1,1)

In [101]:
y_test.shape

(378563, 1)

In [102]:
r2_score(y_test,y_pred)

0.09009812299966236

In [ ]:
0.008